In [14]:
# Creating the SQLite DB + loading CSVs

import sqlite3
import pandas as pd
import numpy as np

DB_PATH = "readmission.db"

# adjust paths if needed
X_path = "data/X_discharge_v1.csv"
y_path = "data/y_readmit30_v1.csv"
ids_path = "IDS_mapping.csv"


# load CSVs
X = pd.read_csv(X_path)
y = pd.read_csv(y_path)
ids = pd.read_csv(ids_path)

# for the join jey. the shared ID that connects the two tables
X["row_id"] = np.arange(len(X)) # creates a unique row number for every row in X
y["row_id"] = np.arange(len(y))

con = sqlite3.connect(DB_PATH) #  live link to the database file

# write to sqlite (replace each run so it's clean)
X.to_sql("X_discharge", con, if_exists="replace", index=False) # creates a SQL table inside the database. delete the old table and recreate it each run. doesn't add pandas row index as a column in SQL
y.to_sql("y_readmit30", con, if_exists="replace", index=False) # leftmost index is usually not a real feature. just pandas’ internal row numbering.
ids.to_sql("ids_mapping", con, if_exists="replace", index=False)

# quick sanity checks
print(pd.read_sql("SELECT COUNT(*) AS n FROM X_discharge", con)) 
print(pd.read_sql("SELECT COUNT(*) AS n FROM y_readmit30", con))
print(pd.read_sql("SELECT COUNT(*) AS n FROM ids_mapping", con))

con.close()

        n
0  101766
        n
0  101766
    n
0  67


In [15]:
con = sqlite3.connect("readmission.db")

ids = pd.read_csv("IDS_mapping.csv")
ids = ids.rename(columns={ids.columns[0]: "admission_type_id"})
ids.to_sql("ids_mapping", con, if_exists="replace", index=False)

con.close()

In [16]:
# to see columns

import sqlite3, pandas as pd
con = sqlite3.connect("readmission.db")

print(pd.read_sql("PRAGMA table_info(X_discharge);", con).head(20)) # first 20 rows of that schema output. PRAGMA asks the database for information about itself.
print(pd.read_sql("PRAGMA table_info(y_readmit30);", con).head(20))
print(pd.read_sql("PRAGMA table_info(ids_mapping);", con).head(20))

con.close()

    cid                      name     type  notnull dflt_value  pk
0     0                      race     TEXT        0       None   0
1     1                    gender     TEXT        0       None   0
2     2                       age     TEXT        0       None   0
3     3         admission_type_id  INTEGER        0       None   0
4     4  discharge_disposition_id  INTEGER        0       None   0
5     5       admission_source_id  INTEGER        0       None   0
6     6          time_in_hospital  INTEGER        0       None   0
7     7                payer_code     TEXT        0       None   0
8     8         medical_specialty     TEXT        0       None   0
9     9        num_lab_procedures  INTEGER        0       None   0
10   10            num_procedures  INTEGER        0       None   0
11   11           num_medications  INTEGER        0       None   0
12   12         number_outpatient  INTEGER        0       None   0
13   13          number_emergency  INTEGER        0       None

In [17]:
# analysis view (joined table)

import sqlite3, pandas as pd
con = sqlite3.connect("readmission.db")

con.execute("""
CREATE VIEW IF NOT EXISTS cohort AS
SELECT
  x.*,
  y.readmit_30
FROM X_discharge x
JOIN y_readmit30 y
  ON x.row_id = y.row_id;
""")

print(pd.read_sql("SELECT COUNT(*) AS n FROM cohort;", con))
print(pd.read_sql("SELECT * FROM cohort LIMIT 5;", con))

con.close()

        n
0  101766
              race  gender      age  admission_type_id  \
0        Caucasian  Female   [0-10)                  6   
1        Caucasian  Female  [10-20)                  1   
2  AfricanAmerican  Female  [20-30)                  1   
3        Caucasian    Male  [30-40)                  1   
4        Caucasian    Male  [40-50)                  1   

   discharge_disposition_id  admission_source_id  time_in_hospital payer_code  \
0                        25                    1                 1    Missing   
1                         1                    7                 3    Missing   
2                         1                    7                 2    Missing   
3                         1                    7                 2    Missing   
4                         1                    7                 1    Missing   

          medical_specialty  num_lab_procedures  ...  insulin  \
0  Pediatrics-Endocrinology                  41  ...       No   
1             

In [18]:
# confirm target distribution + duplicates:

import sqlite3, pandas as pd
con = sqlite3.connect("readmission.db")

out1 = pd.read_sql("""
SELECT
  COUNT(*) AS n,
  SUM(readmit_30) AS readmit_n,
  AVG(readmit_30) AS readmit_rate
FROM cohort;
""", con)

out2 = pd.read_sql("""
SELECT
  COUNT(*) AS n,
  COUNT(DISTINCT row_id) AS distinct_row_id
FROM cohort;
""", con)

con.close()

out1, out2

(        n  readmit_n  readmit_rate
 0  101766      11357      0.111599,
         n  distinct_row_id
 0  101766           101766)

In [19]:
con = sqlite3.connect("readmission.db")

pd.read_sql("""
SELECT
  COUNT(*) AS n,
  COUNT(DISTINCT row_id) AS distinct_row_id
FROM cohort;
""", con)

con.close()

In [20]:
import sqlite3, pandas as pd

DB_PATH = "readmission.db"

def run_sql(q, db_path=DB_PATH):
    con = sqlite3.connect(db_path)
    df = pd.read_sql(q, con)
    con.close()
    return df

In [21]:
# select: chooses which columns to be in the output
    # aggregates, count(*) as n, how many rows in ech group 


# from cohort: tells SQL which table/view to read rows from

# group by: collapses rows into groups (like pandas groupby)
    # get one output per group
    # if no group by: counts total rows in the table

# order by: sorts the output rows 

# sum(readmit_30) as readmit_rate: mean of 0/1 labels, equals readmission rate 

# left join: keeps all cohort rows, even if mapping (to IDS) is missing. adds a readable description

# preds AS: creates a table result that includes all columns from cohort (select *), plus pred=1 if rule triggers.
# cm AS: builds a confusion matrix counts using sums: sums count how many rows match.
    # tn: pred=0, true=0. fp: pred=1,true=0. fn: pred=0,true=1. tp: pred=1,true=1.
# final select: computs precision, recall, flag_rate. the 1.0*tp forces float divison


In [22]:
# 1: Readmission rate by age group

con = sqlite3.connect("readmission.db")

q1 = pd.read_sql("""
SELECT
  age,
  COUNT(*) AS n,
  AVG(readmit_30) AS readmit_rate
FROM cohort
GROUP BY age
ORDER BY age;
""", con)

con.close()

q1

,age,n,readmit_rate
0,[0-10),161,0.018634
1,[10-20),691,0.057887
2,[20-30),1657,0.142426
3,[30-40),3775,0.112318
4,[40-50),9685,0.106040
5,[50-60),17256,0.096662
6,[60-70),22483,0.111284
7,[70-80),26068,0.117731
8,[80-90),17197,0.120835
9,[90-100),2793,0.110992


In [23]:
# risk spikes in 20-30, then sits around 10%-12% in older ages
# could be real (different mix of admissions/conditions)
# or it could be selection effects (who ends up hospitalized and discharged).

In [33]:
con = sqlite3.connect("readmission.db")
pd.read_sql("PRAGMA table_info(ids_mapping);", con)

,cid,name,type,notnull,dflt_value,pk
0,0,admission_type_id,TEXT,0,None,0
1,1,description,TEXT,0,None,0


In [52]:
# 2: Readmission rate by admission type
con = sqlite3.connect("readmission.db")

q2 = """
SELECT
  COUNT(*) AS n_total,
  SUM(CASE WHEN discharge_disposition_id = 7 THEN 1 ELSE 0 END) AS n_left_ama,
  AVG(CASE WHEN discharge_disposition_id = 7 THEN 1.0 ELSE 0.0 END) AS left_ama_rate,
  AVG(readmit_30) AS overall_readmit_rate,
  AVG(CASE WHEN discharge_disposition_id = 7 THEN readmit_30*1.0 ELSE NULL END) AS readmit_rate_left_ama,
  AVG(CASE WHEN discharge_disposition_id <> 7 THEN readmit_30*1.0 ELSE NULL END) AS readmit_rate_not_left_ama
FROM cohort;
"""
run_sql(q2)

,n_total,n_left_ama,left_ama_rate,overall_readmit_rate,readmit_rate_left_ama,readmit_rate_not_left_ama
0,101766,623,0.006122,0.111599,0.144462,0.111397


In [ ]:
# Leaving AMA is rare (~0.61% of encounters), but it’s associated with a higher 30-day readmission rate: 14.45% vs 11.14% (absolute +3.31 percentage points; ~1.30× higher). 
# Note: AMA sample size is small (n=623), so estimate is noisier than the overall rate.

In [25]:
# 3: Length of stay vs readmission

q3 = """
SELECT
  time_in_hospital,
  COUNT(*) AS n,
  AVG(readmit_30) AS readmit_rate
FROM cohort
GROUP BY time_in_hospital
ORDER BY time_in_hospital;
"""
run_sql(q3).head(20)

,time_in_hospital,n,readmit_rate
0,1,14208,0.081785
1,2,17224,0.099396
2,3,17756,0.106668
3,4,13924,0.118070
4,5,9966,0.120309
5,6,7539,0.125879
6,7,5859,0.128350
7,8,4391,0.142337
8,9,3002,0.137242
9,10,2342,0.143467


In [ ]:
# longer stays → higher readmission risk. not perfectly increasing (sampling noise), but trend is there.

In [26]:
# 4: Readmission by gender

q4 = """
SELECT
  gender,
  COUNT(*) AS n,
  AVG(readmit_30) AS readmit_rate
FROM cohort
GROUP BY gender
ORDER BY readmit_rate DESC;
"""
run_sql(q4)

,gender,n,readmit_rate
0,Female,54708,0.112452
1,Male,47055,0.110615
2,Unknown/Invalid,3,0.000000


In [ ]:
# no meaningful difference

In [35]:
# 5: Readmission by race

q5 = """
SELECT
  race,
  COUNT(*) AS n,
  AVG(readmit_30) AS readmit_rate
FROM cohort
GROUP BY race
ORDER BY readmit_rate DESC;
"""
run_sql(q5)

,race,n,readmit_rate
0,Caucasian,76099,0.112906
1,AfricanAmerican,19210,0.112181
2,Hispanic,2037,0.104075
3,Asian,641,0.101404
4,Other,1506,0.096282
5,None,2273,0.082710


In [ ]:
# some groups have small n (e.g., Asian), so their rates can be noisier.

In [47]:
# 6:  Admission type risk ranking

con = sqlite3.connect("readmission.db")

q6 = pd.read_sql("""
SELECT
  m.admission_type AS admission_type,
  COUNT(*) AS n,
  AVG(c.readmit_30) AS readmit_rate
FROM cohort c
LEFT JOIN admission_type_map m
  ON CAST(c.admission_type_id AS INTEGER) = m.admission_type_id
GROUP BY m.admission_type
ORDER BY readmit_rate DESC;
""", con)

con.close()
q6

,admission_type,n,readmit_rate
0,Physician Referral,53990,0.115225
1,Clinic Referral,18480,0.111797
2,Transfer from another health care facility,5291,0.110754
3,Discharged/transferred to SNF,18869,0.103927
4,Transfer from a Skilled Nursing Facility (SNF),4785,0.103448
5,Discharged/transferred to ICF,10,0.100000
6,Court/Law Enforcement,320,0.084375
7,Emergency Room,21,0.000000


In [ ]:
# Readmission risk ranks cleanly by category.
# Referral-based admissions show higher readmission risk than transfers or emergency room cases, reinforcing that structured admissions often involve patients 
# with ongoing care or underlying patient severity

In [49]:
# 7: labs summary by age band + readmission status 


q7 = """
SELECT
  age,
  COUNT(*) AS n,
  AVG(readmit_30) AS readmit_rate,
  AVG(num_lab_procedures) AS avg_labs
FROM cohort
WHERE age IN ('[60-70)', '[70-80)', '[80-90)', '[90-100)')
GROUP BY age
ORDER BY age;
"""
run_sql(q7)

,age,n,readmit_rate,avg_labs
0,[60-70),22483,0.111284,42.600632
1,[70-80),26068,0.117731,43.157396
2,[80-90),17197,0.120835,44.085015
3,[90-100),2793,0.110992,44.695310


In [ ]:
# Among older patients, average lab utilization increases steadily with age.
# Readmission rates rise from ages 60–90, then level off in the oldest group, suggesting increasing clinical complexity with age but possible survivorship or care-path effects at extreme ages.

In [30]:
# 8: Prior inpatient visits

q8 = """
SELECT
  number_inpatient,
  COUNT(*) AS n,
  AVG(readmit_30) AS readmit_rate
FROM cohort
GROUP BY number_inpatient
ORDER BY number_inpatient;
"""
run_sql(q8).head(25)

,number_inpatient,n,readmit_rate
0,0,67630,0.084371
1,1,19521,0.129245
2,2,7566,0.174333
3,3,3411,0.202873
4,4,1622,0.236128
5,5,812,0.314039
6,6,480,0.345833
7,7,268,0.354478
8,8,151,0.443709
9,9,111,0.423423


In [ ]:
# previous inpatient history shows a strong risk signal for readmission
# however, high counts have small n, so it is unstable there


In [31]:
# 9: Emergency visits vs readmission

q9 = """
SELECT
  number_emergency,
  COUNT(*) AS n,
  AVG(readmit_30) AS readmit_rate
FROM cohort
GROUP BY number_emergency
ORDER BY number_emergency;
"""
run_sql(q9).head(25)

,number_emergency,n,readmit_rate
0,0,90383,0.104743
1,1,7677,0.143546
2,2,2042,0.182664
3,3,725,0.202759
4,4,374,0.307487
5,5,192,0.244792
6,6,94,0.234043
7,7,73,0.260274
8,8,50,0.320000
9,9,33,0.363636


In [ ]:
# more ED usage → higher readmission risk. Another strong “utilization history” signal.

In [32]:
# 10: Diagnoses count vs readmission

q10 = """
SELECT
  number_diagnoses,
  COUNT(*) AS n,
  AVG(readmit_30) AS readmit_rate
FROM cohort
GROUP BY number_diagnoses
ORDER BY number_diagnoses;
"""
run_sql(q10)

,number_diagnoses,n,readmit_rate
0,1,219,0.059361
1,2,1023,0.060606
2,3,2835,0.073721
3,4,5537,0.082536
4,5,11393,0.091547
5,6,10161,0.104124
6,7,10393,0.107669
7,8,10616,0.118124
8,9,49474,0.123802
9,10,17,0.176471


In [ ]:
# more diagnoses → more readmission risk 
# However, most patients have number_diagnoses ≤ 9, and higher values are rare (small n), so rates above 10 diagnoses are noisy.

In [50]:
# 11: Discharge disposition risk

q11 = """
SELECT
  discharge_disposition_id,
  COUNT(*) AS n,
  AVG(readmit_30) AS readmit_rate
FROM cohort
GROUP BY discharge_disposition_id
HAVING n >= 100
ORDER BY readmit_rate DESC;
"""
run_sql(q11)

,discharge_disposition_id,n,readmit_rate
0,28,139,0.366906
1,22,1993,0.276969
2,5,1184,0.208615
3,2,2128,0.160714
4,3,13954,0.146625
5,7,623,0.144462
6,8,108,0.138889
7,4,815,0.127607
8,6,12902,0.126957
9,18,3691,0.124357


In [ ]:
# discharge disposition is one of the strongest risk stratifiers in this dataset. 
# It makes sense clinically because where you’re discharged to often reflects severity/support needs.

In [ ]:
# 12: Admission source risk

q12 = """
SELECT
  admission_source_id,
  COUNT(*) AS n,
  AVG(readmit_30) AS readmit_rate
FROM cohort
GROUP BY admission_source_id
HAVING n >= 100
ORDER BY readmit_rate DESC;
"""
run_sql(q12).head(30)

,admission_source_id,n,readmit_rate
0,3,187,0.155080
1,20,161,0.136646
2,5,855,0.118129
3,7,57494,0.116882
4,1,29565,0.105868
5,17,6781,0.104114
6,9,125,0.104000
7,2,1104,0.100543
8,4,3187,0.096956
9,6,2264,0.093640


In [ ]:
# admission source has signal, but weaker than discharge disposition, and a lot of the extreme values are small-n.

In [ ]:
# 13: Highest-risk specialties (filter small n)

q13 = """
SELECT
  medical_specialty,
  COUNT(*) AS n,
  AVG(readmit_30) AS readmit_rate
FROM cohort
GROUP BY medical_specialty
HAVING n >= 200
ORDER BY readmit_rate DESC
LIMIT 15;
"""
run_sql(q13)

,medical_specialty,n,readmit_rate
0,Hematology/Oncology,207,0.193237
1,Oncology,348,0.189655
2,Nephrology,1613,0.153751
3,PhysicalMedicineandRehabilitation,391,0.153453
4,Surgery-Vascular,533,0.138837
5,Psychiatry,854,0.121780
6,Family/GeneralPractice,7440,0.118683
7,Missing,49949,0.115738
8,InternalMedicine,14635,0.112470
9,Emergency/Trauma,7565,0.111831


In [ ]:
# specialty has signal (esp oncology/nephrology), but missingness is huge

In [ ]:
# 14: Medications count bucketed

q14 = """
SELECT
  CASE
    WHEN num_medications <= 5 THEN '0-5'
    WHEN num_medications <= 10 THEN '6-10'
    WHEN num_medications <= 20 THEN '11-20'
    ELSE '21+'
  END AS meds_bucket,
  COUNT(*) AS n,
  AVG(readmit_30) AS readmit_rate
FROM cohort
GROUP BY meds_bucket
ORDER BY n DESC;
"""
run_sql(q14)

,meds_bucket,n,readmit_rate
0,11-20,52025,0.114599
1,21+,23880,0.127764
2,6-10,20795,0.094494
3,0-5,5066,0.074812


In [ ]:
# monotone-ish trend of increasing risk. probably proxying “complexity/sicker patient”.

In [ ]:
# 15: Insulin status vs readmission

q15 = """
SELECT
  insulin,
  COUNT(*) AS n,
  AVG(readmit_30) AS readmit_rate
FROM cohort
GROUP BY insulin
ORDER BY readmit_rate DESC;
"""
run_sql(q15)

,insulin,n,readmit_rate
0,Down,12218,0.138975
1,Up,11316,0.129905
2,Steady,30849,0.111284
3,No,47383,0.100374


In [ ]:
# insulin-related changes track risk (again: proxy for severity/instability).

In [ ]:
# 16: Metformin status vs readmission

q16 = """
SELECT
  metformin,
  COUNT(*) AS n,
  AVG(readmit_30) AS readmit_rate
FROM cohort
GROUP BY metformin
ORDER BY readmit_rate DESC;
"""
run_sql(q16)

,metformin,n,readmit_rate
0,Down,575,0.120000
1,No,81778,0.115165
2,Steady,18346,0.097133
3,Up,1067,0.082474


In [ ]:
# not as clean as insulin; also some categories have small-ish n.

In [ ]:
# 17: Insulin vs time in hospital

q17 = """
SELECT
  insulin,
  time_in_hospital,
  COUNT(*) AS n,
  AVG(readmit_30) AS readmit_rate
FROM cohort
GROUP BY insulin, time_in_hospital
HAVING n >= 50
ORDER BY insulin, time_in_hospital;
"""
run_sql(q17).head(50)

,insulin,time_in_hospital,n,readmit_rate
0,Down,1,1158,0.111399
1,Down,2,1808,0.129425
2,Down,3,2044,0.130137
3,Down,4,1700,0.158235
4,Down,5,1293,0.129157
5,Down,6,998,0.141283
6,Down,7,839,0.140644
7,Down,8,660,0.165152
8,Down,9,448,0.169643
9,Down,10,355,0.197183


In [ ]:
# Within each insulin group, longer stay generally increases readmit_rate (not perfectly monotone, but shows a trend).

In [ ]:
# 18: Worst-risk segment among simple rules

q18 = """
SELECT
  age,
  CASE WHEN time_in_hospital >= 7 THEN '7+' ELSE '<7' END AS los_bucket,
  CASE WHEN number_inpatient >= 1 THEN 'prior_inpatient' ELSE 'no_prior_inpatient' END AS prior_inp,
  COUNT(*) AS n,
  AVG(readmit_30) AS readmit_rate
FROM cohort
GROUP BY age, los_bucket, prior_inp
HAVING n >= 100
ORDER BY readmit_rate DESC
LIMIT 20;
"""
run_sql(q18)

,age,los_bucket,prior_inp,n,readmit_rate
0,[20-30),<7,prior_inpatient,514,0.317121
1,[30-40),7+,prior_inpatient,233,0.227468
2,[30-40),<7,prior_inpatient,987,0.209726
3,[40-50),7+,prior_inpatient,731,0.201094
4,[50-60),7+,prior_inpatient,1257,0.182975
5,[40-50),<7,prior_inpatient,2422,0.174236
6,[70-80),7+,prior_inpatient,2337,0.172871
7,[60-70),<7,prior_inpatient,5569,0.165918
8,[60-70),7+,prior_inpatient,1857,0.163705
9,[80-90),<7,prior_inpatient,4487,0.159126


In [ ]:
# Prior inpatient visits are a huge risk marker.
# Age band [20–30) here is “high risk in this dataset” conditional on prior inpatient.
# This is not causal. It’s just “who in this cohort tends to return within 30 days”.

In [ ]:
# `19: Approx quartile of time in hospital

q19 = """
WITH ordered AS (
  SELECT
    time_in_hospital,
    ROW_NUMBER() OVER (ORDER BY time_in_hospital) AS rn,
    COUNT(*) OVER () AS n
  FROM cohort
)
SELECT
  MAX(CASE WHEN rn = CAST(0.25*n AS INT) THEN time_in_hospital END) AS q1,
  MAX(CASE WHEN rn = CAST(0.50*n AS INT) THEN time_in_hospital END) AS median,
  MAX(CASE WHEN rn = CAST(0.75*n AS INT) THEN time_in_hospital END) AS q3
FROM ordered;
"""
run_sql(q19)

,q1,median,q3
0,2,4,6


In [ ]:
# 25% of stays are ≤ 2 days
# 50% are ≤ 4 days
# 75% are ≤ 6 days

In [ ]:
# 20: Operational cost metric: false alarms per true catch

q20 = """
WITH preds AS (
  SELECT
    readmit_30,
    CASE
      WHEN number_inpatient >= 1 OR time_in_hospital >= 7 THEN 1
      ELSE 0
    END AS pred
  FROM cohort
),
cm AS (
  SELECT
    SUM(CASE WHEN pred=1 AND readmit_30=0 THEN 1 ELSE 0 END) AS fp,
    SUM(CASE WHEN pred=1 AND readmit_30=1 THEN 1 ELSE 0 END) AS tp
  FROM preds
)
SELECT
  fp,
  tp,
  (1.0*fp)/NULLIF(tp,0) AS fp_per_tp
FROM cm;
"""
run_sql(q20)

,fp,tp,fp_per_tp
0,39839,7019,5.67588


In [ ]:
# For every 1 true readmission you correctly flag, you also flag about 5.7 non-readmissions.
# Operational result of high recall under class imbalance.